In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/austin-animal-center-outcomes/Austin_Animal_Center_Outcomes.csv


Currently studying the pandas library and using this dataset as an opportunity to demonstrate what I've learned.

**Questions I want to answer:**
1. How many total records (animals) are in the dataset?
2. What are the most common animal types?
3. What are the most common breeds for dogs and cats?
4. What are the busiest months for animal intakes?
5. Has the number of adoptions changed over time?
6. What are the most common outcomes for animals?
7. Do certain breeds have higher adoption rates?
8. Do younger animals get adopted more quickly?
9. Are certain zip codes associated with more intakes or adoptions?
10. How long do animals typically stay in the shelter before adoption?

### Step 1: Cleaning
1. Does the dataset have missing values? If so, which columns and how many?
2. Are there duplicate rows? Should they be removed?
3. Are there any inconsistent or unexpected values (e.g., negative ages, incorrect dates)?
4. Do column names follow a consistent format (lowercase, snake_case, etc.)?
## handling missing data
* Should missing values be filled, removed, or left as is?
* If filling missing values, what’s the best approach (mean, median, mode, forward/backward fill)?
* Are missing values concentrated in specific columns or random?

## data type validation
* Are all columns in the correct data type (e.g., dates as datetime, numbers as int/float)?
* Do categorical columns have correct and consistent labels?
* Are numerical columns formatted correctly (e.g., no misplaced commas or currency symbols)?

## standardizing and fixing inconsistencies
* Are animal types (e.g., “Dog” vs. “dog”) consistent?
* Are breed names standardized (e.g., “German Shepherd” vs. “GSD”)?
* Do outcome types and intake types have uniform spelling and categorization?
* Are date formats consistent across the dataset?

## handling outliers and erroneous data
* Are there extreme outliers in numerical columns (e.g., very high ages, negative values)?
* Are there animals with duplicate intake and outcome records?
* Are there any animals with unrealistic age values (e.g., “100 years old” for a dog)?

## date and time processing
* Are intake and outcome dates properly formatted and sorted?
* Are there cases where the outcome date is before the intake date?
* Are there unexpected time gaps between intake and outcome?

## final checks
* Does the cleaned dataset retain all necessary columns for analysis?
* Have you documented the cleaning steps for reproducibility?
* Have you saved the cleaned dataset for further analysis?

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        dataset = os.path.join(dirname, filename)

df = pd.read_csv(dataset)
df_backup = df.copy()
df.info()

/kaggle/input/austin-animal-center-outcomes/Austin_Animal_Center_Outcomes.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172113 entries, 0 to 172112
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         172113 non-null  object
 1   Name              122845 non-null  object
 2   DateTime          172113 non-null  object
 3   MonthYear         172113 non-null  object
 4   Date of Birth     172113 non-null  object
 5   Outcome Type      172071 non-null  object
 6   Outcome Subtype   78842 non-null   object
 7   Animal Type       172113 non-null  object
 8   Sex upon Outcome  172111 non-null  object
 9   Age upon Outcome  172106 non-null  object
 10  Breed             172113 non-null  object
 11  Color             172113 non-null  object
dtypes: object(12)
memory usage: 15.8+ MB


Changing data to correct datatypes

In [3]:
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df['MonthYear'] = pd.to_datetime(df['MonthYear'],  errors='coerce')
df['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce')

<ipython-input-3-a1bf5901da9c>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
<ipython-input-3-a1bf5901da9c>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['MonthYear'] = pd.to_datetime(df['MonthYear'],  errors='coerce')


In [4]:
print(df['DateTime'].isna().sum(), df['MonthYear'].isna().sum(), df['Date of Birth'])
df['MonthYear'].describe()

0 0 0        2023-03-25
1        2017-05-02
2        2017-07-12
3        2019-08-16
4        2015-10-08
            ...    
172108   2024-12-16
172109   2021-03-03
172110   2019-02-27
172111   2025-01-06
172112   2014-03-05
Name: Date of Birth, Length: 172113, dtype: datetime64[ns]


count                           172113
mean     2018-10-22 01:03:43.701870336
min                2013-10-01 00:00:00
25%                2016-02-01 00:00:00
50%                2018-07-01 00:00:00
75%                2021-07-01 00:00:00
max                2025-03-01 00:00:00
Name: MonthYear, dtype: object

In [5]:
df['DateTime'].describe()

count                           172113
mean     2018-11-06 11:28:53.957923072
min                2013-10-01 09:31:00
25%                2016-02-01 08:56:00
50%                2018-07-21 17:54:00
75%                2021-07-25 17:47:00
max                2025-03-07 13:48:00
Name: DateTime, dtype: object

In [6]:
df['Date of Birth'].describe()

count                           172113
mean     2016-10-04 09:36:05.923550208
min                1991-09-22 00:00:00
25%                2014-03-11 00:00:00
50%                2016-10-21 00:00:00
75%                2019-11-06 00:00:00
max                2028-05-01 00:00:00
Name: Date of Birth, dtype: object

### 1. Does the dataset have missing values? If so, which columns and how many?
We see below that Outcome Type, Outcome Subtype, Sex upon Outcome, and Age upon Outcome all have missing values.

In [7]:
df.describe()

,DateTime,MonthYear,Date of Birth
count,172113,172113,172113
mean,2018-11-06 11:28:53.957923072,2018-10-22 01:03:43.701870336,2016-10-04 09:36:05.923550208
min,2013-10-01 09:31:00,2013-10-01 00:00:00,1991-09-22 00:00:00
25%,2016-02-01 08:56:00,2016-02-01 00:00:00,2014-03-11 00:00:00
50%,2018-07-21 17:54:00,2018-07-01 00:00:00,2016-10-21 00:00:00
75%,2021-07-25 17:47:00,2021-07-01 00:00:00,2019-11-06 00:00:00
max,2025-03-07 13:48:00,2025-03-01 00:00:00,2028-05-01 00:00:00


## 2. Are there duplicate rows? Should they be removed?
25 duplicated rows. Yes, they should be removed.

In [8]:
df.duplicated().sum()

25

In [9]:
df.drop_duplicates()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A882831,*Hamilton,2023-07-01 18:12:00,2023-07-01,2023-03-25,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White
1,A794011,Chunk,2019-05-08 18:20:00,2019-05-01,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
2,A776359,Gizmo,2018-07-18 16:02:00,2018-07-01,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
3,A821648,NaN,2020-08-16 11:38:00,2020-08-01,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
4,A720371,Moose,2016-02-13 17:59:00,2016-02-01,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
...,...,...,...,...,...,...,...,...,...,...,...,...
172108,A922949,*Scout,2025-03-07 11:45:00,2025-03-01,2024-12-16,Adoption,Foster,Dog,Neutered Male,2 months,Chihuahua Shorthair Mix,Buff/White
172109,A925959,NaN,2025-03-07 11:24:00,2025-03-01,2021-03-03,Transfer,Partner,Dog,Intact Male,4 years,Dachshund/Chihuahua Shorthair,Black/White
172110,A925235,*Gertrude,2025-03-07 12:48:00,2025-03-01,2019-02-27,Adoption,NaN,Dog,Spayed Female,6 years,Miniature Schnauzer,Black/Black
172111,A924228,*Penny Lane,2025-03-07 12:41:00,2025-03-01,2025-01-06,Adoption,NaN,Dog,Spayed Female,1 month,Australian Cattle Dog/Pit Bull,Blue Tick


Are there any inconsistent or unexpected values (e.g., negative ages, incorrect dates)?

In [10]:
df['Name'].value_counts()

Name
Luna        748
Max         721
Bella       674
Rocky       490
Daisy       464
           ... 
*Brocade      1
Babydoll      1
A849756       1
Karati        1
Trueno        1
Name: count, Length: 29585, dtype: int64

In [11]:
df['Name'].isna().sum()
df[df['Name'].isna()]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
3,A821648,NaN,2020-08-16 11:38:00,2020-08-01,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
5,A674754,NaN,2014-03-18 11:47:00,2014-03-01,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
8,A902098,NaN,2024-07-18 12:07:00,2024-07-01,2024-04-10,Adoption,Foster,Cat,Neutered Male,3 months,Domestic Shorthair,Blue Tabby
13,A888689,NaN,2023-09-14 09:22:00,2023-09-01,2020-09-11,Transfer,Snr,Cat,Intact Male,3 years,Domestic Medium Hair,Brown Tabby
14,A684617,NaN,2014-07-27 09:00:00,2014-07-01,2012-07-26,Transfer,SCRP,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black
...,...,...,...,...,...,...,...,...,...,...,...,...
172105,A925579,NaN,2025-03-07 11:18:00,2025-03-01,2023-02-26,Transfer,Partner,Dog,Intact Male,2 years,Shih Tzu,White/Black
172106,A922138,NaN,2025-03-07 11:14:00,2025-03-01,2024-01-24,Adoption,Foster,Cat,Spayed Female,1 year,Domestic Shorthair,Tortie
172107,A925987,NaN,2025-03-07 11:16:00,2025-03-01,2024-03-03,Transfer,Partner,Dog,Intact Male,1 year,Lhasa Apso,Black/Brown
172109,A925959,NaN,2025-03-07 11:24:00,2025-03-01,2021-03-03,Transfer,Partner,Dog,Intact Male,4 years,Dachshund/Chihuahua Shorthair,Black/White


In [12]:
df['Name'].fillna('Unknown', inplace=True)

<ipython-input-12-c9c58b5c4e5b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Name'].fillna('Unknown', inplace=True)


Checking for null values again after using default 'Unknown' name

In [13]:
df['Name'].isna().sum()
df[df['Name'].isna()]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color


Looking at some of the weird names

In [14]:
unique_values = df['Name'].value_counts()
unique_values[unique_values == 1].index

Index(['*Pearl The Girl', 'Navada', '*170 G', 'Tatianna', '*Keroppi',
       'A837241', '*Cat Benatar', '*Ol' Saint Nick', 'Big Maximus', 'Oddie',
       ...
       'Panny', 'Hestia', 'A872166', 'A863086', 'Chicago', '*Brocade',
       'Babydoll', 'A849756', 'Karati', 'Trueno'],
      dtype='object', name='Name', length=17910)

Checking for non-Alphabetical characters

In [15]:
weird_names = df[df['Name'].str.contains(r'[^a-zA-Z*\s]', na=False)]
print(weird_names['Name'])

28                  A878380
37                  A886502
55                  A851472
72                  A839936
98           *Boot Scootin'
                ...        
171661         *Mr. Pickles
171708         *Mr. Pickles
171946           *Choco-Cat
172075       *Mae(Deceased)
172080    24-1809C Snowball
Name: Name, Length: 5484, dtype: object


Why are there two cats named *Mr. Pickles? Same cat or different?

In [16]:
df[df['Name'] == '*Mr. Pickles']

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
11235,A772703,*Mr. Pickles,2019-01-16 13:55:00,2019-01-01,2015-05-22,Adoption,Foster,Dog,Neutered Male,3 years,American Pit Bull Terrier Mix,Blue/White
171661,A741366,*Mr. Pickles,2017-01-05 17:25:00,2017-01-01,2015-12-31,Adoption,NaN,Dog,Neutered Male,1 year,Norwich Terrier Mix,Tan
171708,A741366,*Mr. Pickles,2025-02-23 11:51:00,2025-02-01,2015-12-31,Adoption,NaN,Dog,Neutered Male,9 years,Norwich Terrier Mix,Tan


In [17]:
df['DateTime'].describe()

count                           172113
mean     2018-11-06 11:28:53.957923072
min                2013-10-01 09:31:00
25%                2016-02-01 08:56:00
50%                2018-07-21 17:54:00
75%                2021-07-25 17:47:00
max                2025-03-07 13:48:00
Name: DateTime, dtype: object

In [18]:
df['MonthYear'].value_counts()

MonthYear
2019-06-01    2244
2014-07-01    2084
2019-08-01    1979
2015-06-01    1976
2019-07-01    1949
              ... 
2021-01-01     649
2020-05-01     442
2021-02-01     422
2020-04-01     393
2025-03-01     213
Name: count, Length: 138, dtype: int64

In [19]:
df['Date of Birth'].value_counts()

Date of Birth
2016-05-01    121
2014-05-05    120
2015-09-01    117
2014-04-21    117
2018-04-16    108
             ... 
2001-02-02      1
1999-01-04      1
1997-07-07      1
2006-08-09      1
2000-04-22      1
Name: count, Length: 8613, dtype: int64

In [20]:
df['Outcome Type'].value_counts()

Outcome Type
Adoption           83709
Transfer           48264
Return to Owner    25529
Euthanasia         10686
Died                1658
Rto-Adopt           1230
Disposal             868
Missing               92
Relocate              28
Stolen                 5
Lost                   2
Name: count, dtype: int64

In [21]:
df['Outcome Subtype'].value_counts()

Outcome Subtype
Partner                40046
Foster                 17729
Rabies Risk             4843
Suffering               4094
Snr                     4041
SCRP                    3211
Out State                931
In Kennel                866
Aggressive               611
Offsite                  512
In Foster                413
Medical                  352
At Vet                   351
Field                    238
Behavior                 176
Enroute                  112
Court/Investigation       99
Emergency                 52
Underage                  37
In Surgery                33
Customer S                20
Prc                       20
Possible Theft            16
Barn                      16
In State                  12
Emer                      11
Name: count, dtype: int64

In [22]:
df['Animal Type'].value_counts()

Animal Type
Dog          93718
Cat          68652
Other         8839
Bird           870
Livestock       34
Name: count, dtype: int64

In [23]:
df['Sex upon Outcome'].value_counts()

Sex upon Outcome
Neutered Male    60363
Spayed Female    54784
Intact Male      22056
Intact Female    21614
Unknown          13294
Name: count, dtype: int64

In [24]:
df['Age upon Outcome'].value_counts()

Age upon Outcome
1 year       28510
2 years      27603
2 months     23069
3 years       9702
3 months      9137
1 month       7332
4 months      6010
4 years       5766
5 years       5217
5 months      4404
6 months      4162
6 years       3403
7 years       2995
8 years       2938
3 weeks       2871
2 weeks       2868
8 months      2689
4 weeks       2407
10 months     2382
10 years      2367
7 months      2257
9 months      1754
9 years       1642
1 weeks       1338
12 years      1154
11 months     1096
1 week         952
11 years       920
13 years       740
3 days         565
14 years       533
2 days         523
1 day          506
15 years       461
4 days         352
6 days         341
0 years        268
5 days         221
5 weeks        202
16 years       200
17 years       107
18 years        64
19 years        29
20 years        26
-1 years         6
22 years         6
-3 years         2
25 years         2
21 years         1
24 years         1
28 years         1
30 years      

Let's calculate it manually using 'DateTime' and 'Date of Birth'

In [25]:
new_age_series = (df['DateTime'] - df['Date of Birth'])
new_age_series.describe()

count                          172113
mean      763 days 01:52:48.034372760
std      1043 days 16:54:17.678271008
min              -1548 days +11:53:00
25%                  90 days 07:16:00
50%                 369 days 17:44:00
75%                 812 days 18:56:00
max               10996 days 15:07:00
dtype: object

The negative years do not make sense.

In [26]:
new_age_series.min()

Timedelta('-1548 days +11:53:00')

In [27]:
df['Breed'].value_counts()

Breed
Domestic Shorthair Mix                           34026
Domestic Shorthair                               23197
Pit Bull Mix                                     10113
Labrador Retriever Mix                            8734
Chihuahua Shorthair Mix                           6901
                                                 ...  
Australian Kelpie/Bichon Frise                       1
Cavalier Span/Border Collie                          1
Duck/American                                        1
English Springer Spaniel/Cardigan Welsh Corgi        1
Doberman Pinsch/Alaskan Husky                        1
Name: count, Length: 2990, dtype: int64

In [28]:
df['Color'].value_counts()

Color
Black/White        17820
Black              14537
Brown Tabby        10587
Brown               7068
White               6076
                   ...  
Blue Tabby/Buff        1
Chocolate/Gold         1
Torbie/Blue            1
Torbie/Calico          1
Cream/Chocolate        1
Name: count, Length: 661, dtype: int64

In [29]:
df.describe()

,DateTime,MonthYear,Date of Birth
count,172113,172113,172113
mean,2018-11-06 11:28:53.957923072,2018-10-22 01:03:43.701870336,2016-10-04 09:36:05.923550208
min,2013-10-01 09:31:00,2013-10-01 00:00:00,1991-09-22 00:00:00
25%,2016-02-01 08:56:00,2016-02-01 00:00:00,2014-03-11 00:00:00
50%,2018-07-21 17:54:00,2018-07-01 00:00:00,2016-10-21 00:00:00
75%,2021-07-25 17:47:00,2021-07-01 00:00:00,2019-11-06 00:00:00
max,2025-03-07 13:48:00,2025-03-01 00:00:00,2028-05-01 00:00:00
